# Data Collection

### Import Libraries For Data Collection

In [ ]:
from tapsdk import TapSDK, TapInputMode
from tapsdk.models import AirGestures
import pandas as pd
import numpy as np
import keyboard
import time
import os 
import glob

tap_instance = []
tap_identifiers = []
global x_array 
global y_array 

modes = ["text", "controller", "controller_text", "raw"]

### Functions

In [ ]:
class Functionality:
    current_x = None
    current_y = None
    
    # For connection notification
    def on_connect(self, identifier, name, fw):
        print("Tap has connected")
        print(identifier + " Tap: " + str(name), " FW Version: ", fw)
        if identifier not in tap_identifiers:
            tap_identifiers.append(identifier)
        print("Connected taps:")
        for identifier in tap_identifiers:
            print(identifier)

    # For disconnection notification
    def on_disconnect(self, identifier):
        print("Tap has disconnected")
        if identifier in tap_identifiers:
            tap_identifiers.remove(identifier)
        for identifier in tap_identifiers:
            print(identifier)

    # For mouse data
    def on_mouse_event(self, identifier, dx: int, dy: int, isMouse):
        if isMouse:
            self.current_x = dx 
            self.current_y = dy
            # print(str(dx), str(dy))
        else:
            pass
            # print("Air: ", str(dx), str(dy))

    #For adding columns on top 
    def add_top_column(self, df: pd.DataFrame, top_col: str, inplace=False):
        if not inplace:
            df = df.copy()
        
        df.columns = pd.MultiIndex.from_product([[top_col], df.columns])
        return df

### Data Collection

#### Non-Automated Collection

In [ ]:
def main(): 
    count = 0 
    start = input(f"Type 'ST' to start running")
    
    while start == "ST":
        global tap_instance
        tap_instance = TapSDK()
        fun = Functionality()
        tap_instance.run()

        # tap_instance.register_connection_events(on_connect)
        # tap_instance.register_disconnection_events(on_disconnect)
        tap_instance.register_mouse_events(fun.on_mouse_event)
        tap_instance.set_input_mode(TapInputMode(modes[2]))   

        trial = True 
        while trial:
            if(keyboard.is_pressed("q")):
                trial = False
                print("You have chosen to end this trial.")
            else: 
                if str(fun.current_x) != "None":
                    # print(str(fun.current_x) + " "+ str(fun.current_y) + " " + "got it")
                    with open(f"Data\\Random_+1\\random_{count}.txt", "a") as my_file:
                        my_file.write(str(fun.current_x) + " " + str(fun.current_y) +"\n")
                pass

        count += 1
        if  count <= 100:  
            start = input("Type 'ST' to keep running")
        else: 
            start = "Finish"
            print(start)

if __name__ == "__main__":
    main() 

#### Data Cleaning and Dumping

In [ ]:
# Remove repetitive lines from both CW and CCW text files 
path1 = r"C:\Users\MoRevolution\Desktop\College\Classes\2022-2023\J-term\CSC-380\Tap Strap Functionality\Data\Random"
listing = glob.glob(path1 + '\\*.txt')
count = 0 
for file in listing:
    lines_seen = set() # holds lines already seen
    outfile = open(f"Data\\Cleaned_Random\\random_{count}.txt", "a")
    for line in open(file, "r"):
        if line not in lines_seen: # not a duplicate
            outfile.write(line)
            lines_seen.add(line)
    outfile.close()
    count += 1 

In [ ]:
#Add data from text files to csv
trial_df = pd.DataFrame()
count = 0 
path1 = r"C:\Users\MoRevolution\Desktop\College\Classes\2022-2023\J-term\CSC-380\Tap Strap Functionality\Data\Cleaned_Random"
listing = glob.glob(path1 + '\\*.txt')
for file in listing:
    x_array = np.array([])
    y_array = np.array([])
    # con = input("Press Y")
    with open(file, 'r') as my_file: 
        for line in my_file:
            line = line.split(" ")
            x_array = np.append((x_array), [int(line[0])])
            y_array = np.append((y_array), [int(line[1])])
    trial_column = pd.DataFrame()
    trial_column[f"x_{count}"] = x_array.tolist()
    trial_column[f"y_{count}"] = y_array.tolist()
    trial_df= pd.concat([trial_df, trial_column],axis=1)
    count += 1 
print("Finished all!!")
trial_df.to_csv(input("Type file name"), index=False)

#### Experimental Automated Collection

In [ ]:
def main(): 
    trial_df = pd.DataFrame()
    count = 0
    start = input(f"Type 'St' to start running")

    while start == "St":
        trial = input(f"Type 'ST' to start running this trial")
        x_array = np.array([])
        y_array = np.array([])
        while trial == "ST":
            print("We are on trial" + str(count))

            # instantiate a Tap object and start registering events
            global tap_instance
            tap_instance = TapSDK()
            fun = Functionality()
            tap_instance.run()
            tap_instance.register_mouse_events(fun.on_mouse_event)

            #use 1 or 3 for controller or raw data mode respectively in list modes
            tap_instance.set_input_mode(TapInputMode(modes[1]))   

            # Command to end the recording of one trail within a case
            if(keyboard.is_pressed("q")):
                trail = "END"
                print("You have chosen to end this trial.")
            else:          
                #append each position change value to the array 
                x_array = np.append(x_array,[fun.current_x])
                y_array = np.append(x_array,[fun.current_y])   
                print(str(fun.current_x), str(fun.current_y)) 
                pass

            # Command to end the recording of trials within this database
        if (keyboard.is_pressed('p')): 
            start = "End"
            print("You have chosen to end this iteration of trials. Saving data...")
        else:  
            #Add this trial to the trial_df dataframe
            trial_column = pd.DataFrame([[x_array, y_array]], columns=['x_'+ str(count), 'y_'+ str(count)])
            # trial_column = add_top_column(trial_column, "Trial_" + str(count))
            trial_df= pd.concat([trial_df, trial_column])

            print(f"Trial {count} just ended, starting next trial. ")
            count += count
            print(f"Trial {count} is next")
            pass

    trial_df.to_csv(input("Type in the name for this csv file"), index=False)

if __name__ == "__main__":
    main()

In [ ]:
def main():
    trial_df = pd.DataFrame()
    count = 0
    start = input(f"Type 'St' to start running")

    while start == "St":
        trial = input(f"Type 'ST' to start running this trial")
        x_array = np.array([])
        y_array = np.array([])
        while trial == "ST":
            print("We are on trial" + str(count))

            # instantiate a Tap object and start registering events
            global tap_instance
            tap_instance = TapSDK()
            tap_instance.run()
            tap_instance.register_connection_events(on_connect)
            tap_instance.register_disconnection_events(on_disconnect)
            tap_instance.register_mouse_events(on_mouse_event)
            
            #append each position change value to the array 
            x_array = np.append(x_array,[dx])
            y_array = np.append(x_array,[dy])   
            print(str(dx), str(dy)) 

            #use 1 or 3 for controller or raw data mode respectively in list modes
            tap_instance.set_input_mode(TapInputMode(modes[1])) 

            # Command to end the recording of one trail within a case
            if(keyboard.is_pressed("q")):
                trail = "END"
                print("You have chosen to end this trial.")
            else: 
                pass
        # Command to end the recording of trials within this database
        if (keyboard.is_pressed('p')): 
            start = "End"
            print("You have chosen to end this iteration of trials. Saving data...")
        else:  
            #Add this trial to the trial_df dataframe
            trial_column = pd.DataFrame([[x_array, y_array]], columns=['x_'+ str(count), 'y_'+ str(count)])
            # trial_column = add_top_column(trial_column, "Trial_" + str(count))
            trial_df= pd.concat([trial_df, trial_column])

            print(f"Trial {count} just ended, starting next trial. ")
            count += count
            print(f"Trial {count} is next")
            pass

    trial_df.to_csv(input("Type in the name for this csv file"), index=False)

if __name__ == "__main__":
    main()